reference papers for genre annotation:  
https://www.tagtraum.com/download/schreiber_msdgenre_ismir2015.pdf  

# Data loading

In [1]:
import pandas as pd
import numpy as np

In [2]:
song_artist_genre = pd.read_csv('./data/song_artist_genre.csv')

In [3]:
check = song_artist_genre['song_id'].duplicated()

In [4]:
check.index[check==True]

Int64Index([   354,    355,    356,    491,    492,    493,    821,    822,
               823,   1685,
            ...
            931241, 931242, 931243, 934813, 961799, 961800, 961801, 966143,
            966144, 966145],
           dtype='int64', length=2876)

Some songs are duplicated. Needs to be dropped.

In [5]:
song_artist_genre.drop_duplicates(subset=['song_id'], inplace=True)

In [6]:
song_artist_genre.head()

,artist_7digitalid,artist_familiarity,artist_hotttnesss,artist_id,artist_latitude,artist_location,artist_longitude,artist_mbid,artist_name,artist_playmeid,...,genre_13,confidence_13,genre_14,confidence_14,genre_15,confidence_15,genre_16,confidence_16,genre_17,confidence_17
0,4069,0.649822,0.394032,ARYZTJS1187B98C555,NaN,NaN,NaN,357ff05d-848a-44cf-b608-cb34b5701ae5,Faster Pussy cat,44895,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,113480,0.439604,0.356992,ARMVN3U1187FB3A1EB,NaN,NaN,NaN,8d7ef530-a6fd-4f8f-b2e2-74aec765e0f9,Karkkiautomaatti,-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,63531,0.643681,0.437504,ARGEKB01187FB50750,55.8578,"Glasgow, Scotland",-4.24251,3d403d44-36ce-465c-ad43-ae877e65adc4,Hudson Mohawke,-1,...,Instrumental,0.005725,Dance,0.003817,Dance & DJ,0.003817,Ambient Funk Pop,0.003817,Glitch,0.003817
3,65051,0.448501,0.372349,ARNWYLR1187B9B2F9C,NaN,NaN,NaN,12be7648-7094-495f-90e6-df4189d68615,Yerba Brava,34000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,158279,0.000000,0.000000,AREQDTE1269FB37231,NaN,NaN,NaN,NaN,Der Mystic,-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
song_artist_genre.columns

Index(['artist_7digitalid', 'artist_familiarity', 'artist_hotttnesss',
       'artist_id', 'artist_latitude', 'artist_location', 'artist_longitude',
       'artist_mbid', 'artist_name', 'artist_playmeid', 'audio_md5',
       'duration', 'end_of_fade_in', 'key', 'key_confidence', 'loudness',
       'mode', 'mode_confidence', 'release', 'release_7digitalid',
       'song_hotttnesss', 'song_id', 'song_title', 'start_of_fade_out',
       'tempo', 'time_signature', 'time_signature_confidence', 'title',
       'track_7digitalid', 'track_id', 'main_genre', 'n_taggers', 'genre_1',
       'confidence_1', 'genre_2', 'confidence_2', 'genre_3', 'confidence_3',
       'genre_4', 'confidence_4', 'genre_5', 'confidence_5', 'genre_6',
       'confidence_6', 'genre_7', 'confidence_7', 'genre_8', 'confidence_8',
       'genre_9', 'confidence_9', 'genre_10', 'confidence_10', 'genre_11',
       'confidence_11', 'genre_12', 'confidence_12', 'genre_13',
       'confidence_13', 'genre_14', 'confidence_14', '

In [8]:
user_song = pd.read_csv('./data/train_triplets_filtered_ratings.csv')

In [9]:
user_song.head()

,user_id,song_id,play_count,rating
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1,1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,2
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1,1


In [10]:
song_artist_genre.loc[song_artist_genre['main_genre']!=song_artist_genre['genre_1']][['song_id', 'main_genre', 'genre_1']]

,song_id,main_genre,genre_1
0,SOQMMHC12AB0180CB8,NaN,NaN
3,SOBNYVR12A8C13558C,NaN,NaN
4,SOHSBXH12A8C13B0DF,NaN,NaN
5,SOZVAPQ12A8C13B63C,NaN,NaN
8,SOPMIYT12A6D4F851E,NaN,NaN
...,...,...,...
1001925,SOWCNSN12AB018070F,Electronic,Electronica
1001926,SOVRJOO12A8C143C7A,NaN,NaN
1001928,SOXQYIQ12A8C137FBB,NaN,NaN
1001929,SOHODZI12A8C137BB3,NaN,NaN


Dataset needs to be cleaned: some songs don't have genres labeled

In [18]:
song_genre = song_artist_genre[~song_artist_genre['main_genre'].isna()]
song_genre = song_genre[['song_hotttnesss', 'song_id', 'song_title','title', 
       'track_7digitalid', 'track_id', 'main_genre', 'n_taggers', 'genre_1', 
       'confidence_1', 'genre_2', 'confidence_2', 'genre_3', 'confidence_3', 
       'genre_4', 'confidence_4', 'genre_5', 'confidence_5', 'genre_6', 
       'confidence_6', 'genre_7', 'confidence_7', 'genre_8', 'confidence_8', 
       'genre_9', 'confidence_9', 'genre_10', 'confidence_10', 'genre_11', 
       'confidence_11', 'genre_12', 'confidence_12', 'genre_13', 
       'confidence_13', 'genre_14', 'confidence_14', 'genre_15', 
       'confidence_15', 'genre_16', 'confidence_16', 'genre_17', 
       'confidence_17']]

According to the reference paper, the author created subgenre trees for some "seed-genres" and used subgenres for annotation to produce mutiple labels for each song.

In [19]:
song_genre.head()[['main_genre', 'n_taggers', 'genre_1', 
       'confidence_1', 'genre_2', 'confidence_2', 'genre_3', 'confidence_3', 
       'genre_4', 'confidence_4']]

,main_genre,n_taggers,genre_1,confidence_1,genre_2,confidence_2,genre_3,confidence_3,genre_4,confidence_4
1,Pop,1.0,Pop,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
2,Electronica/Dance,524.0,Electronica/Dance,0.446565,Electronic,0.316794,Electronica,0.059160,RPM,0.032443
6,Reggae,14.0,Reggae,0.928571,Dancehall Reggae,0.071429,NaN,NaN,NaN,NaN
7,Hip-Hop,102.0,Hip-Hop,0.333333,Rap,0.294118,Hip-Hop/Rap,0.205882,Other Rap/Hip-Hop,0.039216
9,Cadence,1.0,Cadence,1.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
song_genre.shape

(676155, 42)

# Song profiles

In [21]:
counts = song_genre['genre_1'].value_counts()
counts

Rock                         110538
Pop                           48884
Jazz                          33608
Electronic                    27182
Country                       20360
                              ...  
Indierock, New Wave, Punk         1
5. Dance (synthpop)               1
Homestead                         1
Polish Jazz                       1
Pop/80's/New Wave                 1
Name: genre_1, Length: 6937, dtype: int64

Some genres don't have enough songs. Select the top genres that have at least 1000 songs as our features, which produces 65 genres.

In [22]:
genres = counts[counts>1000].index.tolist()

Pre-populate the values for genres as 0. They will be replaced by the confidence values later.

In [23]:
old_columns = ['genre_1', 
       'confidence_1', 'genre_2', 'confidence_2', 'genre_3', 'confidence_3', 
       'genre_4', 'confidence_4', 'genre_5', 'confidence_5', 'genre_6', 
       'confidence_6', 'genre_7', 'confidence_7', 'genre_8', 'confidence_8', 
       'genre_9', 'confidence_9', 'genre_10', 'confidence_10', 'genre_11', 
       'confidence_11', 'genre_12', 'confidence_12', 'genre_13', 
       'confidence_13', 'genre_14', 'confidence_14', 'genre_15', 
       'confidence_15', 'genre_16', 'confidence_16', 'genre_17', 
       'confidence_17']

In [24]:
def fill_confidence(x, genre):
    for i in range(0, len(old_columns), 2):
        if x[old_columns[i]] == genre:
            return x[old_columns[i+1]]
    
    return 0

In [25]:
from tqdm import tqdm

for genre in tqdm(genres):
    song_genre[genre] = song_genre.apply(lambda x: fill_confidence(x, genre), axis=1)

100%|██████████████████████████████████████████████████████████████████████████████████| 63/63 [27:48<00:00, 26.49s/it]


In [26]:
song_genre.drop(columns=old_columns, inplace=True)
song_genre.to_csv("./data/songs_genres.csv", index=False)

# User profiles

In [28]:
user_song.shape

(32525889, 4)

In [29]:
user_genre = pd.DataFrame(user_song['user_id'].unique(), columns=['user_id'])

In [31]:
user_genre.shape

(558350, 1)

In [32]:
def create_profile(user_id):
    user_set = user_song.loc[user_song['user_id']==user_id]
    user_set_genre = pd.merge(
        left=user_set, right=song_genre, left_on="song_id", right_on="song_id", how="left"
)
    user_set_genre[genres] = user_set_genre[genres].multiply(user_set_genre['play_count'], axis='index')

    return user_set_genre[genres].sum(axis=0)/user_set_genre['play_count'].sum()

Not enough computing power to generate profiles for all users. Approximately 13 days needed to generate all user profiles

In [33]:
#tqdm.pandas()
#user_genre[genres]=user_genre['user_id'].progress_apply(create_profile)

In [47]:
# select the top 1000 frequent users to generate sample user profiles
user_subset = user_song['user_id'].value_counts()[:1000].index.tolist()

In [50]:
user_genre_subset=user_genre.loc[user_genre['user_id'].isin(user_subset)].copy()

tqdm.pandas()
user_genre_subset[genres]=user_genre_subset['user_id'].progress_apply(create_profile)

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [29:35<00:00,  1.78s/it]


In [51]:
user_genre_subset.to_csv("./data/example_subset_users_genres.csv", index=False)

# Approach 1: Similarity between User and Songs

In [34]:
from sklearn.metrics.pairwise import cosine_similarity

In [35]:
def recommend(user_id, k):
    user_profile = create_profile(user_id)
    diff = cosine_similarity(create_profile(user_id).values.reshape((1, -1)), song_genre[genres])
    return song_genre[['song_id','song_title']].iloc[np.argsort(-diff)[0][:k]]

In [36]:
user_id = 'b64cdd1a0bd907e5e00b39e345194768e330d652'
recommend(user_id, 3)

,song_id,song_title
818036,SOOHRSR12A81C22C58,All Man's Land
334906,SOTMMAZ12AB01859E8,One Above One
58129,SOIGQZV12A6D4F7424,Fly Me Away (Ladytron Remix)


For this approach, the results are the recommended songs only and there're no ground truth. So evaluation is not available.

# Approach 2: Predict Ratings

First drop songs that don't have genre labels

In [37]:
user_song=user_song.loc[user_song['song_id'].isin(song_genre['song_id'])]

The original dataset is too large and correlation matrix cannot be computed. A subset is chosen to demonstrate the custom KNN algo

In [38]:
# choose the top 1000 most listened songs
song_subset= user_song['song_id'].value_counts()[:1000].index.tolist()

In [39]:
song_genre_subset=song_genre.loc[song_genre['song_id'].isin(song_subset)]

In [40]:
user_song_subset=user_song.loc[user_song['song_id'].isin(song_subset)]

In [41]:
from surprise import accuracy, KNNBasic, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise.model_selection import GridSearchCV

Code reference: Surprise open-source GitHub repo  
https://github.com/NicolasHug/Surprise/tree/687ed960ef8dac599b3c49f1cf7d6ad1e5bad1f4/surprise

In [42]:
class CustomKNN(KNNBasic):
    def __init__(self, k=40, min_k=1, sim_options={}, verbose=True, **kwargs):      
        # Always call base method before doing anything.
        KNNBasic.__init__(self, k=40, min_k=1, sim_options={}, verbose=True, **kwargs)
        
        # We're customizing the item-item similarities, so the default needs to be cahged to False
        self.sim_options["user_based"] = False

    def compute_similarities(self):
        """Build the similarity matrix.
        Replace the user-item utility matrix with the song-genre matrix
        Returns:
            The similarity matrix."""
        
        global song_genre_subset
        m = song_genre_subset[['song_id']+genres].set_index('song_id')
        sim = np.asarray(m.T.corr())
        return sim       

In [43]:
reader = Reader(rating_scale=(1, 5))

data = Dataset.load_from_df(user_song_subset[['user_id', 'song_id', 'rating']], reader=reader)

In [44]:
trainset, testset = train_test_split(data, test_size=0.2)

algo = CustomKNN(random_state=42)

algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 0.9778


0.9777632623653745

In [45]:
param_grid = {"k": [10, 20, 40, 50], "min_k": [1, 2], 'random_state':[42]}
gs = GridSearchCV(CustomKNN, param_grid, measures=["rmse"], cv=5)

gs.fit(data)

# best RMSE score
print(gs.best_score["rmse"])

# combination of parameters that gave the best RMSE score
print(gs.best_params["rmse"])

0.9803423653340758
{'k': 10, 'min_k': 1, 'random_state': 42}


In [46]:
algo_tuned = CustomKNN(k=10, min_k=1, random_state=42)

algo_tuned.fit(trainset)
predictions = algo_tuned.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 0.9778


0.9777632623653745